# SPGD_empiar11415

In [ ]:
#%pip install opencv-python

In [ ]:
#%pip install mrcfile

In [ ]:
#%pip install scikit-image

In [ ]:
#%pip install matplotlib

In [ ]:
#%pip install "information_theory @ git+https://github.com/vicente-gonzalez-ruiz/information_theory"

In [ ]:
#%pip install gdown

In [ ]:
#%pip install "denoising @ git+https://github.com/vicente-gonzalez-ruiz/denoising"
!rm denoising
!ln -s ../../../src/denoising denoising

In [ ]:
import logging
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
#from ipywidgets import *
import cv2
import time
#import kernels
import skimage
#from skimage import io as skimage_io
import mrcfile
import logging
import information_theory
from collections import namedtuple

In [ ]:
logging.basicConfig(format="[%(filename)s:%(lineno)s %(funcName)s()] %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
vol_filename = "empiar11415.mrc"

In [ ]:
Args = namedtuple("args", ["input", "output"])
fn, fe = vol_filename.split(".")
args = Args(vol_filename , fn + "_denoised." + fe)
print(args)

In [ ]:
%%bash
ln -sf /nas/vruiz/vols/empiar11415.mrc .

In [ ]:
#noisy = skimage.io.imread(args.input, plugin="tifffile").astype(np.float32)
stack_MRC = mrcfile.open(args.input)
noisy = stack_MRC.data

In [ ]:
Z_dim = noisy.shape[0]
Z2 = Z_dim//2

In [ ]:
plt.imshow(noisy[Z2], cmap="gray")

In [ ]:
def get_gaussian_kernel(sigma=1):
    number_of_coeffs = 3
    number_of_zeros = 0
    while number_of_zeros < 2 :
        delta = np.zeros(number_of_coeffs)
        delta[delta.size//2] = 1
        coeffs = scipy.ndimage.gaussian_filter1d(delta, sigma=sigma)
        number_of_zeros = coeffs.size - np.count_nonzero(coeffs)
        number_of_coeffs += 1
    return coeffs[1:-1]

std_dev = 2.0
sigma = np.array([std_dev, std_dev, std_dev])
kernel = [None]*3
kernel[0] = get_gaussian_kernel(sigma[0])
kernel[1] = get_gaussian_kernel(sigma[1])
kernel[2] = get_gaussian_kernel(sigma[2])

In [ ]:
import denoising.volume.OF_gaussian as SPGD

In [ ]:
denoiser = SPGD.Monochrome_Denoising(logger)

In [ ]:
denoised = denoiser.filter(noisy, kernel, l=2, w=5)

In [ ]:
with mrcfile.new(f"denoised.mrc", overwrite=True) as mrc:
    mrc.set_data(denoised.astype(np.float32))
    mrc.data

In [ ]:
def read_MRC(file_path):
    return mrcfile.read(file_path)

In [ ]:
denoised = read_MRC("denoised.mrc")

In [ ]:
figure(figsize=(32, 32))
plt.subplot(1, 3, 1)
plt.title("original")
slice_idx = noisy.shape[0]//2
imgplot = plt.imshow(noisy[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 2)
title = f"OF Gaussian $\\sigma={std_dev}$"
plt.title(title)
plt.imshow(denoised[slice_idx][::-1, :], cmap="gray")
plt.subplot(1, 3, 3)
plt.title("difference")
plt.imshow(noisy[slice_idx][::-1, :] - denoised[slice_idx][::-1, :], cmap="gray")
plt.show()

In [ ]:
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 0:500, 0:500], cmap="gray")

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(16, 16))
slice_idx = denoised.shape[0]//2
plt.imshow(denoised[slice_idx, 0:500, 0:500], cmap="gray")
plt.savefig("denoised.pdf", bbox_inches='tight')

In [ ]:
from my_google_auth import DriveHandler
service = DriveHandler.get_drive_service()
handler = DriveHandler.DriveHandler(service)
MY_SHARED_DRIVE_ID = '1hGHvkP46fxLCQbUlyYhAS_eVl6PollQM'

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoised.pdf',
    drive_file_name='empiar11415__denoised__SPGD.pdf',
    drive_folder_id=MY_SHARED_DRIVE_ID
)

In [ ]:
figure(figsize=(16, 16))
slice_idx = noisy.shape[0]//2
plt.imshow(noisy[slice_idx, 200:600, 200:600], cmap="gray")

In [ ]:
fig_single, ax_single = plt.subplots(figsize=(10, 10))
ax_single.imshow(denoised[slice_idx][::-1, :], cmap="gray")
ax_single.set_title(title)
fig_single.savefig(title + ".png")
plt.close(fig_single)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[1]//2
axs[0].imshow(noisy[:, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised")
fig.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 32))
slice_idx = noisy.shape[2]//2
axs[0].imshow(noisy[:, :, slice_idx], cmap="gray")
axs[0].set_title(f"Noisy")
axs[1].imshow(denoised[:, :, slice_idx], cmap="gray")
axs[1].set_title(f"Denoised")
fig.tight_layout()
plt.show()

In [ ]:
stack_MRC = mrcfile.open("denoised.mrc")
denoised = stack_MRC.data

In [ ]:
slice_idx = noisy.shape[0] // 2

fig, axes = plt.subplots(1, 2, figsize=(20, 20))

# Plot the original slice z
im1 = axes[0].imshow(noisy[slice_idx, 0:512, 0:512].T, cmap='gray', origin='lower')
axes[0].set_title(f'Original Slice Z={slice_idx}')
axes[0].grid(False)

# Plot the original slice z+1
im2 = axes[1].imshow(denoised[slice_idx, 0:512, 0:512].T, cmap='gray', origin='lower')
axes[1].set_title(f'N2N-Odd-Even-Registered Denoised Slice Z={slice_idx}')
axes[1].grid(False)

plt.tight_layout()
plt.show()

In [ ]:
uploaded_file_id = handler.upload(
    local_file_path='denoised.mrc',
    drive_file_name='empiar11415__denoised__SPGD.mrc',
    drive_folder_id=MY_SHARED_DRIVE_ID
)